# 9符合Python风格的对象
## 9.1对象表现形式
1. 对象字符串表示形式
    + repr()——对象的_\_repr__方法，以便于开发者理解的方式返回对象的字符串表示形式。
    + str()——对象的_\_str__方式，以便于用户理解的方式返回对象的字符串表示形式。
    + bytes()——对象的_\_bytes__方法，生成实例的二进制表达形式


In [3]:

from array import array
import math


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        """ 实现迭代方法，把vector2d变成可迭代对象 """
        return (i for i in (self.x, self.y))

    def __repr__(self):
        """ *self返回x,y变量给format，前提是实现__iter__方法 """
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR2D_V1
    @classmethod  # <1>
    def frombytes(cls, octets):  # <2>
        typecode = chr(octets[0])  # <3>
        memv = memoryview(octets[1:]).cast(typecode)  # typecode=d，二进制转换成浮点型
        return cls(*memv)  # <5>
# END VECTOR2D_V1

In [4]:
v1 = Vector2d(3,4)
print(repr(v1))
v1_clone = eval(repr(v1))## 通过字符串形式克隆Vector2d实例
print(v1 == v1_clone)

Vector2d(3.0, 4.0)
True


In [7]:
print(bytes(v1))
v1_clone = Vector2d.frombytes(bytes(v1))## 通过字节形式克隆Vector2d实例
print(v1_clone == v1)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
True


## 9.4 classmethod 与 staticmethod
1. classmethod装饰器，定义操作类，该方法第一个参数时类本身，不是实例。常用于定义备选构造方法
2. staticmethod装饰器，静态方法类似普通的函数，只不过在类的定义体中

In [5]:
class Demo:
    @classmethod
    def classmeth (*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args
print(Demo.classmeth())
print(Demo.classmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


## 9.5 格式化显示
1. 格式化方法：
    + 内置函数format(obj, format_spec)
    + 字符串方法str.format()
    + 基于各个类型相应的._\_format__(format_spec)方法实现；format_space——格式说明符
2. [格式规范微语言](https://docs.python.org/3/library/string.html#format-specification-mini-language)
    + b——二进制，x——十六进制，f——浮点数，%——百分数
    + 格式规范微语言是可扩展的，各个类可以自行决定如何解释format_spec参数，由_\_format__方法实现。如datetime
3. 在自定义的格式代码中选择字母，避免使用其他类型已使用过的字母。整数使用的代码有"bcdoxxn’，浮点数使用的代码有"eEfFgGn%’，字符串使用的代码有"s’

In [9]:
print(format(1/2.43,'0.4f'))
print('{rate:0.2f}'.format(rate=1/2.43))#{rate:0.2f}冒号:左边是字段名，右边是格式说明符（格式规范微语言）
print(format(111,'b'))
print(format(111,'x'))
print(format(0.5,'.1%'))

0.4115
0.41
1101111
6f
50.0%


In [17]:
from datetime import datetime
from time import strftime
now = datetime.now()
print(format(now, '%Y-%m-%d %H:%M:%S'))
print(strftime('%Y-%m-%d %H:%M:%S'))

2022-09-22 23:11:09
2022-09-22 23:11:09


## 9.6 可散列
1. 类实现\_\_hash\_\_、\_\_eq\_\_方法
2. 类的属性求只读（标记为私有self.\_\_x），__@property__ 装饰器把读值方法标记为特性


In [14]:
from array import array
import math

# BEGIN VECTOR2D_V3_PROP
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)  # <1>标记为私有属性
        self.__y = float(y)

    @property  # <2>把读值方法标记为特性 即vector.x调用该函数
    def x(self):  # <3>与属性同名
        return self.__x  # <4>

    @property  # <5>
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <6>

    # remaining methods follow (omitted in book listing)
# END VECTOR2D_V3_PROP

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

# BEGIN VECTOR_V3_HASH
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
# END VECTOR_V3_HASH

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):#自定义格式代码，获取向量极坐标
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


In [15]:
v1 = Vector2d(3, 4)
v1_clone = Vector2d.frombytes(bytes(v1))

In [ ]:
"""测试样例
A 2-dimensional vector class

    >>> v1 = Vector2d(3, 4)
    >>> print(v1.x, v1.y)
    3.0 4.0
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector2d(3.0, 4.0)
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector2d(0, 0))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector2d.frombytes(bytes(v1))
    >>> v1_clone
    Vector2d(3.0, 4.0)
    >>> v1 == v1_clone
    True


Tests of ``format()`` with Cartesian coordinates:

    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of the ``angle`` method::

    >>> Vector2d(0, 0).angle()
    0.0
    >>> Vector2d(1, 0).angle()
    0.0
    >>> epsilon = 10**-8
    >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
    True
    >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
    True


Tests of ``format()`` with polar coordinates:

    >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector2d(1, 1), '.3ep')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector2d(1, 1), '0.5fp')
    '<1.41421, 0.78540>'

# BEGIN VECTOR2D_V3_DEMO
Tests of `x` and `y` read-only properties:

    >>> v1.x, v1.y
    (3.0, 4.0)
    >>> v1.x = 123
    Traceback (most recent call last):
      ...
    AttributeError: can't set attribute

# END VECTOR2D_V3_HASH_DEMO

Tests of hashing:
# BEGIN VECTOR2D_V3_HASH_DEMO

    >>> v1 = Vector2d(3, 4)
    >>> v2 = Vector2d(3.1, 4.2)
    >>> hash(v1), hash(v2)
    (7, 384307168202284039)
    >>> len(set([v1, v2]))
    2

# END VECTOR2D_V3_DEMO

"""

## 9.7 Python私有属性，受保护属性
1. 
    私有属性使用场景：举个例子。有人编写了一个名为 Dog 的类，这个类的内部用到了 mood实例属性，但是没有将其开放。现在，你创建了 Dog 类的子类：Beagle。如果你在毫不知情的情况下又创建了名为 mood 的实例属性，那么在继承的方法中就会把 Dog 类的 mood 属性覆盖掉。这是个难以调试的问题。
    名称改写(name mangling)：为了避免这种情况，如果以_mood 的形式（两个前导下划线，尾部没有或最多有一个下划线）命名实例属性，Python 会把属性名存入实例的_dict_属性中，而且会在前面加上一个下划线和类名。因此，对Dog 类来说，_mood 会变成_Dog_mood；对 Beagle 类来说，会变成Beaglemood。


In [4]:
v1 = Vector2d(3,4)
print(v1.__dict__)
v1._Vector2d__x = 1
print(v1._Vector2d__x)#2. 私有化的属性不可直接修改，需要通过该方式进行修改。由此可知该方式并不能使变量真正实现私有化
v1.x = 3
print(v1.x)


{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}
1


AttributeError: can't set attribute

## 9.8 使用\_\_slots\_\_类属性节省空间
1. 默认情况下，Python 在各个实例中名为_dict__的字典里存储实例属性。但字典会消耗大量的内存，当处理的实例数量很大，且属性不多，可使用\_\_slots\_\_类属性，节省内存。原理是让解释器在元组中存储实例属性，而不是字典
2. 定义方法：
```class Vector2d:
    __slots__ = ('__x', '__y')

    typecode = 'd'

    # methods follow (omitted in book listing)
# END VECTOR2D_V3_SLOTS

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
```
3. 注意在类中定义\_\_slots\_\_属性后不可有slots中所列名称之外的其他属性

## 9.9覆盖类属性
1. Python 中类属性可以为实例属性提供默认值：
    + 如vector2d有一个typecode类属性，并非实例属性，使用self.typecode方式默认读取的是类属性。
    + 如果为不存在的实例属性赋值，会新建实例属性。假如我们为typecode 实例属性赋值，那么同名类属性不受影响。即self.typecode=x，创建了一个实例属性，后续读取该值也是x而非类属性。不影响别的实例